In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import os
import time
import re
import random
from tqdm.notebook import tqdm
#from user_agents import 

In [31]:
def generate_proxy():
    proxy_url = "https://us-proxy.org/"
    proxy_list = []
    r = requests.get(proxy_url)
    #r.status_code
    soup = BeautifulSoup(r.text)
    table = soup.find("table")
    trs = table.find_all("tr")
    ths = table.find_all("th")
    for tr in trs[1:]:
        proxy_dict = {}
        for i, td in enumerate(tr.find_all("td")):
            proxy_dict[ths[i].text] = td.text
        proxy_list.append(proxy_dict)
    
    proxy_list.remove({})
    return pd.DataFrame(proxy_list)

In [32]:
generate_proxy()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,89.187.177.89,80,US,United States,anonymous,no,no,7 seconds ago
1,138.68.60.8,8080,US,United States,anonymous,no,no,7 seconds ago
2,89.187.177.106,80,US,United States,anonymous,no,no,7 seconds ago
3,89.187.177.98,80,US,United States,anonymous,no,no,7 seconds ago
4,89.187.177.90,80,US,United States,anonymous,no,no,3 minutes ago
...,...,...,...,...,...,...,...,...
195,23.94.9.209,3128,US,United States,transparent,no,no,3 hours 41 minutes ago
196,193.56.64.28,8085,US,United States,transparent,no,no,3 hours 41 minutes ago
197,93.177.117.227,8085,US,United States,transparent,no,no,3 hours 41 minutes ago
198,85.209.149.112,8085,US,United States,transparent,no,no,3 hours 41 minutes ago


In [70]:
class Total_Wine_Scraper():
    def __init__(self):
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
        self.total_wine_url = "https://www.totalwine.com"
        self.digit_re = re.compile(r"\d+")
        self.wine_details_l = []
    
    
    def get_hrefs(self, soup):
        anchors = soup.find("div", attrs = {"view":"tiles"}).find_all("a")
        hrefs = [self.total_wine_url + a.get("href") for a in anchors]
        clean_urls = [href.replace("&tab=3","") for href in hrefs]
        return list(set(clean_urls))
    
    def get_last_page_num(self, soup):
        nums = []
        for li in soup.find_all("li", attrs={"role":"none"}):
            text= li.text
            if text != "":
                try:
                    nums.append(int(text))
                except:
                    pass
        return max(nums)
    
    
        
    def scrape_wine_details(self, link):
        detail_dict = {}
        time.sleep(1+random.randint(1,5))
        r = requests.get(link, headers= self.headers)
        print("Status code: {}".format(r.status_code))
        soup = BeautifulSoup(r.text)
        wine_name = soup.h1.text
        
        detailsTPI = self.find_detailsTableProductInfo(soup)
        table_rows = soup.find("div", class_= detailsTPI)
        for table_row in table_rows:
            detail_val = table_row.span#.text
            detail_key = table_row.text#.replace(detail_val,"")
            detail_dict[detail_key] = detail_val
            
        product_red_badge = self.find_product_red_badge(soup)
        red_badge = soup.find("div", class_= product_red_badge)
        rb_num = self.digit_re.findall(red_badge.text)[-1]
        rb_source = red_badge.text.replace(rb_num,"")
        detail_dict["red_badge_num"] = rb_num
        return detail_dict

        
    
    def find_detailsTableProductInfo(self,soup):
        """
        returns the class string that has detailsTableProductInfo
        """
        divs = soup.find_all("div")
        for div in divs:
            classes = div.get("class")
            if classes != None:
                for c in classes:
                    if "detailsTableProductInfo" in c:
                        return c
    def find_product_red_badge(self, soup):
        divs = soup.find_all("div")
        for div in divs:
            classes = div.get("class")
            if classes != None:
                for c in classes:
                    if "product-red-badge" in c:
                        return c
    def scrape_wines(self):
        total_wine_page_url = "https://www.totalwine.com/wine/c/c0020?&page=1&pageSize=24&aty=1,0,0,1"
        r = requests.get(total_wine_page_url, headers = self.headers)
        soup = BeautifulSoup(r.text)
        last_num = self.get_last_page_num(soup)
        time.sleep(random.randint(1,5))
        # scraping each page
        
        page_pbar = tqdm(total = last_num, desc="Scraping page")
        for i in range(1,last_num+1):
            total_wine_fstring = "https://www.totalwine.com/wine/c/c0020?&page={}&pageSize=24&aty=1,0,0,1".format(i)
            # pausing to avoid being ip banned
            time.sleep(5+random.randint(-3,5))
            r = requests.get(total_wine_fstring, headers = self.headers)
            soup = BeautifulSoup(r.text)
            wine_urls = self.get_hrefs(soup)
            url_pbar = tqdm(len(wine_urls))
            for wine_url in wine_urls:
                self.wine_details_l.append(self.scrape_wine_details(wine_url))
                url_pbar.update(1)
            
            page_pbar.update(1)
            
        return pd.DataFrame(self.wine_details_l)
    

In [71]:
tws = Total_Wine_Scraper()
tws.scrape_wines()

Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 403


AttributeError: 'NavigableString' object has no attribute 'span'

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
total_wine_url = "https://www.totalwine.com/"
r = requests.get(total_wine_url, headers= headers)
print(r.status_code)

200


In [34]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
tws = Total_Wine_Scraper()
r = requests.get("https://www.totalwine.com/wine/c/c0020?&page=1&pageSize=24&aty=1,0,0,1", headers = headers)
soup = BeautifulSoup(r.text)

In [37]:
hrefs = tws.get_hrefs(soup)
new_hrefs= []

for href in hrefs:
    new_hrefs.append(href.replace("&tab=3",""))
    
print(len(hrefs),len(set(new_hrefs)))

72 24


In [9]:
driver = webdriver.Chrome(executable_path="tmp_src/chromedriver")
driver.get(total_wine_url)

In [10]:
wine_nav = driver.find_element_by_id("AT_WineNav")
wine_nav.click()

In [3]:
total_wine_wine_url = "https://www.totalwine.com/wine/c/c0020?&page=1&pageSize=24&aty=1,0,0,1"
r = requests.get(total_wine_wine_url, headers=headers)
r.status_code

200

In [4]:
soup = BeautifulSoup(r.text)

In [19]:
soup.find("div", attrs={"view":"tiles"}).find_all("a")[0].get("href")

'/wine/deals/red-wine/red-blend/renieri-invetro/p/113710750?s=1108&igrules=true'

In [6]:
wine_r = requests.get(hs[0], headers=headers)
wine_r.status_code

200

In [7]:
wine_soup = BeautifulSoup(wine_r.text)

In [8]:
wine_soup.find_all("h1")

[<h1 class="productTitle__3XDd9UVh">RENIERI INVETRO, 2016</h1>,
 <h1 class="productTitle__3XDd9UVh">RENIERI INVETRO, 2016</h1>]

In [11]:
wine_divs = wine_soup.find_all("div")

In [16]:
detail_tablelabels= re.compile(r"\detailsTableLabel")

In [26]:
for wine_div in wine_divs:
    classes = wine_div.get("class")
    if classes != None:
        for c in classes:
            if "detailsTableProductInfo" in c:
                print(c)

detailsTableProductInfo__2_nFoPxO


In [33]:
 wine_soup.find("div", class_= "detailsTableProductInfo__2_nFoPxO").find_all("div")[0].span

<span>Renieri</span>

In [34]:
for wine_div in wine_divs:
    classes = wine_div.get("class")
    if classes != None:
        for c in classes:
            if "product-red-badge" in c:
                print(c)

product-red-badge__2JqIvCXw


In [45]:
re.compile(r"\d+").findall('92James Suckling')

['92', '012']

In [47]:
wine_soup.find("div", class_= "product-red-badge__2JqIvCXw")

<div class="product-red-badge__2JqIvCXw"><div class="badgeContainer__36P-nTb2" style="background-image:url(/product/c426ca81aac7ea509e8f3edf320e6830.png)"><div class="redBadgeNumber__DZXSWqnj">92</div></div><div class="redBadgeSource__1hMXdJ5Z">James Suckling</div></div>

In [46]:
'92James Suckling9'.replace("92","")

'James Suckling9'

In [50]:
for li in soup.find_all("li", attrs={"role":"none"}):
    print(re.compile(r"\d+").findall(li.text))
    print("\n\n\n")

[]




['1']




['2']




['3']




['4']




[]




['1054']




[]






In [ ]:
def total_wine_scraper():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
    total_wine_url = "https://www.totalwine.com/"
    total_wine_wine_url = "https://www.totalwine.com/wine/c/c0020?&page=1&pageSize=24&aty=1,0,0,1"
    
